In [13]:
import pickle

with open('problem_batch_mu_sig_std.pkl', "rb") as f:
    batch = pickle.load(f)

In [15]:
import torch

# Check if there are any zeros in log_var
# Check if there are any zeros in the tensor
tensor = batch["std"]
has_zeros = (tensor == 0).any()

print("Are there any zeros in the tensor?", has_zeros)

# Find the indices of zeros
if has_zeros:
    zero_indices = torch.nonzero(tensor == 0, as_tuple=True)
    print("Indices of zeros:", zero_indices)

Are there any zeros in the tensor? tensor(True, device='cuda:0')
Indices of zeros: (tensor([16, 30], device='cuda:0'), tensor([13, 13], device='cuda:0'))


In [12]:
# Check for very close to 0
import torch

# Example tensor
tensor = batch['log_var']

# Define the threshold
threshold = 1e-3

# Check if there are any elements smaller than the threshold
close_to_zero = (tensor.abs() < threshold).any()

print("Are there any elements close to zero?", close_to_zero)

# Find the indices of elements close to zero
if close_to_zero:
    close_to_zero_indices = torch.nonzero(tensor.abs() < threshold, as_tuple=True)
    print("Indices of elements close to zero:", close_to_zero_indices)

Are there any elements close to zero? tensor(True, device='cuda:0')
Indices of elements close to zero: (tensor([ 4,  7, 19], device='cuda:0'), tensor([223, 160,  53], device='cuda:0'))


In [4]:
import torch
torch.min(batch["z"])

tensor(-inf, device='cuda:0', grad_fn=<MinBackward1>)

In [14]:
import torch
torch.set_printoptions(threshold=20_000)
print(batch["pred_"])

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
      